In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

2024-04-16 13:11:45.205547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 13:11:45.266532: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 13:11:45.267300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 13:11:46.179199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
church_test = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/test/church/images"
lizard_test = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/test/lizard/images"
church_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/church/images"
pond_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/pond/images"
lizard_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/lizard/images"
dioscuri_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/dioscuri/images"
cup_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/transp_obj_glass_cup/images"
cylinder_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/transp_obj_glass_cylinder/images"
temple_train = "/datalake/datasets/ML_Davis/image-matching-challenge-2024/train/multi-temporal-temple-baalshamin/images"

In [3]:
def create_binary_classification_cnn(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [4]:
def load_images_and_labels(base_dirs, target_size=(128, 128)):
    images = []
    labels = []
    for base_dir in base_dirs:
        #print(f"Processing directory: {base_dir}")
        if os.path.isdir(base_dir):
            for file_name in sorted(os.listdir(base_dir)):
                file_path = os.path.join(base_dir, file_name)
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    #print(f"Loading image: {file_path}")
                    img = image.load_img(file_path, target_size=target_size)
                    img_array = image.img_to_array(img)
                    images.append(img_array)
                    label = 1 if 'church' in base_dir else 0
                    labels.append(label)
        else:
            print(f"Not a directory: {base_dir}")
    if not images:
        print("No images found after processing. Check the file paths and extensions.")
    else:
        print(f"Loaded {len(images)} images successfully.")
    return np.array(images), np.array(labels)

In [5]:
# Directory paths
train_dirs = [church_train, lizard_train, pond_train, dioscuri_train, cup_train, cylinder_train, temple_train]
test_dirs = [church_test, lizard_test]

# Load training and testing images
train_images, train_labels = load_images_and_labels(train_dirs)
test_images, test_labels = load_images_and_labels(test_dirs)

# Calculate steps per epoch and validation steps
steps_per_epoch = max(len(train_images) // 20, 1)  # Ensure at least one step
validation_steps = max(len(test_images) // 20, 1)  # Ensure at least one step

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=20, shuffle=True)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=20, shuffle=False)


Loaded 485 images successfully.
Loaded 90 images successfully.


In [6]:
# Model setup
model = create_binary_classification_cnn((128, 128, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
if steps_per_epoch > 0:
    model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10)

# Evaluate model
if validation_steps > 0:
    results = model.evaluate(test_generator, steps=validation_steps)
    print("Test loss, test acc:", results)
else:
    print("Validation steps issue: check the division for batch size and total samples.")


Epoch 1/10


2024-04-16 13:13:37.727528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 5s 177ms/step - loss: 0.5740 - accuracy: 0.7118
Epoch 2/10
24/24 [==============================] - 4s 161ms/step - loss: 0.3203 - accuracy: 0.8387
Epoch 3/10
24/24 [==============================] - 4s 164ms/step - loss: 0.4078 - accuracy: 0.7204
Epoch 4/10
24/24 [==============================] - 4s 162ms/step - loss: 0.2056 - accuracy: 0.9097
Epoch 5/10
24/24 [==============================] - 4s 156ms/step - loss: 0.2045 - accuracy: 0.9097
Epoch 6/10
24/24 [==============================] - 4s 157ms/step - loss: 0.1546 - accuracy: 0.9441
Epoch 7/10
24/24 [==============================] - 4s 160ms/step - loss: 0.1380 - accuracy: 0.9398
Epoch 8/10
24/24 [==============================] - 4s 161ms/step - loss: 0.0839 - accuracy: 0.9548
Epoch 9/10
24/24 [==============================] - 4s 157ms/step - loss: 0.2132 - accuracy: 0.9140
Epoch 10/10
24/24 [==============================] - 4s 168ms/step - loss: 0.2047 - accuracy: 0.9247


2024-04-16 13:14:18.267647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 0s 17ms/step - loss: 0.2860 - accuracy: 0.8500
Test loss, test acc: [0.28596049547195435, 0.8500000238418579]


In [7]:
# Predict on the test data
predictions = model.predict(test_generator, steps=validation_steps)
print(predictions)

4/4 [==============================] - 0s 23ms/step


2024-04-16 13:14:18.563602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[[9.6756595e-01]
 [9.7401100e-01]
 [8.6042982e-01]
 [9.4873935e-01]
 [8.6156619e-01]
 [6.3106710e-01]
 [7.5897712e-01]
 [8.8132846e-01]
 [6.1045891e-01]
 [8.2249540e-01]
 [7.9059720e-01]
 [7.1347845e-01]
 [7.7578276e-01]
 [8.8971031e-01]
 [9.0936321e-01]
 [8.1185114e-01]
 [7.1082056e-01]
 [9.8231411e-01]
 [9.8290390e-01]
 [9.6987508e-02]
 [7.4109620e-01]
 [3.9534122e-01]
 [6.3964516e-01]
 [7.3865622e-01]
 [8.6163753e-01]
 [9.6369362e-01]
 [8.5740674e-01]
 [9.6330231e-01]
 [6.3528764e-01]
 [8.5449725e-01]
 [9.7560602e-01]
 [2.8992406e-01]
 [5.2230424e-01]
 [7.2852367e-01]
 [9.0885156e-01]
 [8.7251848e-01]
 [4.0637156e-01]
 [5.1369476e-01]
 [9.2329091e-01]
 [8.1138444e-01]
 [7.8278559e-01]
 [3.5546422e-05]
 [4.4756535e-02]
 [1.2536579e-03]
 [6.2441942e-03]
 [7.3904760e-02]
 [6.4186436e-01]
 [1.5138403e-01]
 [2.2808143e-03]
 [5.4273206e-01]
 [5.3505880e-01]
 [5.5792279e-13]
 [8.1712049e-15]
 [2.7987664e-11]
 [2.8328485e-11]
 [1.5976817e-11]
 [6.4011204e-13]
 [4.0910684e-02]
 [7.1285486e-0

In [16]:
# Single image prediction example
img_path = 'bothalf.png'
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print(str(prediction[0]) + " The image is classified as a Church.")
else:
    print(str(prediction[0]) + " The image is not classified as a Church.")

1/1 [==============================] - 0s 26ms/step
[0.9992271] The image is classified as a Church.


In [10]:
# Telling machine that it is wrong (retraining on single image prediction)
def retrain_model_on_feedback(model, img_path, actual_label, learning_rate=0.01):
    # Load image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    actual_label = np.array([actual_label])

    # Train the model on this one instance where machine was wrong
    model.fit(img_array, actual_label, epochs=1)

# Example of using the function
img_path = 'upsidedown.png'
actual_label = 1  # Correct label if the prediction was wrong
retrain_model_on_feedback(model, img_path, actual_label)

1/1 [==============================] - 1s 922ms/step - loss: 1.6962 - accuracy: 0.0000e+00
